In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import optuna

/Users/hamsterlord/miniforge3/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv("data/final/final.csv")
df = df[df['pot_doc'] != '[]']

y = np.array(df['documentid'])
X = np.array(df)

# df["pot_doc"].astype("O")
# df["pot_date"].astype("O")
# df["pot_price"].astype("O")
# df["pot_name"].astype("O")
# df["pot_address"].astype("O")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
x_column = list(df.columns)
x_column.remove('documentid')

X_train_df = pd.DataFrame(data = X_train, columns = df.columns)
X_test_df = pd.DataFrame(data = X_test, columns = df.columns)
y_train_df = pd.DataFrame(data = y_train, columns = ['documentid'])
y_test_df = pd.DataFrame(data = y_test, columns = ['documentid'])

X_train_df= X_train_df.drop(columns="documentid")
X_test_df= X_test_df.drop(columns="documentid")

In [8]:
def compute_accuracy(x, y, z,a):
    results = []
    for row in X_train_df.iterrows():
        if (len(eval(row[1]['pot_doc']))==0):
            continue
        dists = []
        
        for idx in range(len(eval(row[1]['pot_doc']))):
            dists.append((x*eval(row[1]['pot_price'])[idx])**2 
            + (y*(100-eval(row[1]['pot_date'])[idx])/30)**2
            + (z*(100-eval(row[1]['pot_address'])[idx])/30)**2
            #+ (a*eval(row[1]['pot_name'])[idx])**2
            )
        dists = np.array(dists)
        results.append(eval(row[1]['pot_doc'])[dists.argmin()])
    results = np.array(results)
    return np.sum(results == np.array(y_train_df['documentid']))/len(X_train_df)
    
def compute_accuracy2(x, y, z, a):
    results = []
    for row in X_train_df.iterrows():
        if (len(eval(row[1]['pot_doc']))==0):
            continue
        dists = []
        
        for idx in range(len(eval(row[1]['pot_doc']))):
            dists.append((x*eval(row[1]['pot_price'])[idx])**2 
            + (y*(100-eval(row[1]['pot_date'])[idx])/30)**2
            + (z*(100-eval(row[1]['pot_address'])[idx])/30)**2
            #+ (a*eval(row[1]['pot_name'])[idx])**2
            )
        dists = np.array(dists)
        results.append(eval(row[1]['pot_doc'])[dists.argmin()])
    results = np.array(results)
    return results

In [6]:

# dfs = pd.DataFrame()
# ss = pd.DataFrame()

# dfs['result'] = ss.apply(lambda row: (row['documentid'] in row['pot_doc']), axis=1)
# print(dfs)
# num_trues = dfs['result'].value_counts()[True]
# print(num_trues)
def objective(trial):
    x = trial.suggest_float('x', 0, 10)
    #y = trial.suggest_float('y', 0, 10)
    z = trial.suggest_float('z', 0, 50)
    #a = trial.suggest_float('a', 0, 10)
    return 1-compute_accuracy(x,1.0,z,0)

study = optuna.create_study()
study.optimize(objective, n_trials=100)

study.best_params  # E.g. {'x': 2.002108042}

[I 2023-01-29 01:46:03,562] A new study created in memory with name: no-name-7105dab5-5100-4a86-8461-a96c1ad972e9
[I 2023-01-29 01:46:04,286] Trial 0 finished with value: 0.41755319148936165 and parameters: {'x': 8.883180866218025, 'z': 8.931660834393423}. Best is trial 0 with value: 0.41755319148936165.
[I 2023-01-29 01:46:05,089] Trial 1 finished with value: 0.574468085106383 and parameters: {'x': 1.1507959668166312, 'z': 43.70892684136092}. Best is trial 0 with value: 0.41755319148936165.
[I 2023-01-29 01:46:05,904] Trial 2 finished with value: 0.40691489361702127 and parameters: {'x': 3.8262894825577423, 'z': 23.484526851359693}. Best is trial 2 with value: 0.40691489361702127.
[I 2023-01-29 01:46:06,718] Trial 3 finished with value: 0.401595744680851 and parameters: {'x': 6.138463109316238, 'z': 31.28466242148575}. Best is trial 3 with value: 0.401595744680851.
[I 2023-01-29 01:46:07,498] Trial 4 finished with value: 0.39893617021276595 and parameters: {'x': 7.766993080909396, 'z'

{'x': 7.766993080909396, 'z': 27.161816130797224}

In [11]:
res = compute_accuracy2(study.best_params['x'], 1, study.best_params['z'], 0)
real = list(X_train_df['pot_doc'])
print(len(res), len(real))

376 376


In [ ]:
for i in range(len(res)):
    if res[i] != real[i]:
        print(res[i], real[i])